<h2>Team Members</h2>
<ul>
<li>Sohad Hossam Eldin 1190019</li>
<li>Bassant Hisham Mohamed 1190018</li>
<li>Yasmin Hashem Niazy</li>
<li>Mary Ashraf 1190322</li>
</ul>

<h4>Imports</h4>

In [ ]:
import re
from pyarabic.araby import strip_tashkeel

<h4>Reading the dataset</h4>

In [ ]:
dataset_txt = open(r"train.txt", "r").read()
print(type(dataset_txt))

<h4>Cleaning the dataset</h4>

In [ ]:
# do we need to remove [ :ص]
chars_to_remove = r"\(\s*[a-zA-Z0-9_-]+\s*/\s*[a-zA-Z0-9_-]+\s*\)|[a-zA-Z0-9_-]+" 
dataset_cleaned = re.sub(chars_to_remove, "", dataset_txt)
print(dataset_cleaned)

<h4>Segmenting dataset into sentences</h4>

In [ ]:
dataset_sentences = re.split(".",dataset_cleaned)
print(type(dataset_sentences))
print(dataset_sentences)

<h4>Removing Tashkeel</h4>

In [ ]:
dataset_cleaned = strip_tashkeel(dataset_cleaned)
print(dataset_cleaned)